<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>  Modelo Logístico de Clasificación con JAX</center></span>

## <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Camilo José Torres Jiménez, Msc, cjtorresj@unal.edu.co

## <span style="color:blue">Estudiantes auxiliares</span>

1. Daniel Andrés Rojas, anrojasor@unal.edu.co
2. Camilo Chitivo, cchitivo@unal.edu.co
3. Jessica López Mejía, jelopezme@unal.edu.co

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [El modelo lineal de clasificación](#El-modelo-lineal-de-clasificación)
* [Importar los módulos requeridos](#Importar-los-módulos-requeridos)
* [Función de predicción](#Función-de-predicción)
* [Carga del conjunto de datos Iris](#Carga-del-conjunto-de-datos-Iris)
* [Gradiente](#Gradiente)
* [Entrenamiento del modelo](#Entrenamiento-del-modelo)
* [Calculando el valor de la función y el gradiente con value_and_grad](#Calculando-el-valor-de-la-función-y-el-gradiente-con-value_and_grad)


## <span style="color:blue">Introducción</span>

Este código fue tomado y  adaptado de [Google Colab](https://colab.research.google.com/drive/1qNxKmi0QpkunqTDdpXfVLlneG-NFDN9c). En este ejercicio usaremos el famoso conjunto de datos *iris*. Sin embargo no se usaran todos los datos, porque en este ejercicio vamos a introducir el modelo logístico clásico que permite separar en dos clases. Los datos de la primera clase son omitidos y los datos se recodifican para tener solamente dos clases. Próximamente usaremos todos los datos.

## <span style="color:blue">El modelo lineal de clasificación</span>

En este  modelo se tienen varias variables regresoras o explicativas de entrada y una variable dicotómica de salida.

El propósito central es construir un modelo para predecir la probabilidad de que los elementos del espacio de entrada pertenezcan a una de dos clases, las cuales denotaremos como 0 y 1 respectivamente.

Supongamos que tenemos dos variables $X_1$ y $X_2$ que se espera permitan predecir si un elemento del conjunto de entrada pertenece a una clase: clase 1 ($Y=1$) o clase 0 ($Y=0$).

El modelo desde el punto de vista estadístico se escribe como

$$
[Y_i|X_1=x_{i1},X_2=x_{i2}] \sim \text{Bernoulli}(\pi_i),
$$

en donde 

$$
\pi_i = \frac{1}{1 + exp(-[b +w_1x_{i1} + w_2x_{i2})]}, i =i,\cdots,N
$$

En el entrenamiento se encontraran los pesos $w_1,w_2,$ y el intercepto $b$ que minimizan una determinada función de pérdida, a partir de un conjunto de datos de entrenamiento. 


Una vez garantizado que la máquina generaliza bien, probando con los datos de validación, la expresión anterior se utiliza para predecir la probabilidad que un nuevo valor no observado en el espacio de entrada, digamos $(x_1,x_2)$ pertenezca a a una clase. 

Por construcción $\pi$ es la probabilidad que el elemento $x$ pertenezca a la clase 1. Por lo tanto si por ejemplo $\pi = 0.8$ para un elemento, entonces lo clasificamos en la clase 1. 


La idea central que está detrás de este tipo de modelos se puede apreciar en la siguiente imagen.


<figure>
<center>
<img src="../Imagenes/clasificador_lineal.png" width="600" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Clasificador Lineal</p>
</figcaption>
</figure>


Se trata de un clasificador lineal simple. Vamos a suponer que la máquina de aprendizaje ya está entrenada, por lo que los parámetros $w,b$ están fijos.

Observe que la línea roja divide el espacio $\mathcal{R}^2$ en  tres regiones. La primera es justamente la recta, que corresponde a un modelo de regresión como se estudió en la lección de [regresión lineal](am_intro_regresion.ipynb). Sobre la línea se cumple la ecuación 

$$
wx+b =0.
$$

Por otro lado se tiene que si $wx+b=0$, entonces la probabilidad $\pi$ es dada en este caso por

$$
\pi = \frac{1}{1+exp(-(wx+b))} = \frac{1}{2}.
$$

La segunda región está a la derecha. Usted puede verificar que en este caso, para todos los valores de $x$ se tiene que  $wx+b>0$. Como consecuencia, se tiene que $\pi>\tfrac{1}{2}$. en el caso extremo para valores $x$ muy alejados hacia la derecha, se tiene que $wx+b\to \infty$ y en consecuencia $\pi\to 1$.


En la tercera región (a la izquierda) ocurre el comportamiento simétrico pero en el otro sentido. Ahora $wx+b<0$, para todos los valores de $x$.  Se tiene que $\pi<\tfrac{1}{2}$. En el caso extremo para valores $x$ muy alejados hacia la izquierda, se tiene que $wx+b\to -\infty$ y en consecuencia $\pi\to 0$.


### Conclusión

El separador lineal funciona de la siguiente forma en este caso.

1. Si $\pi(x)$ es mayor que 0.5, la clase que debe asigna es 1. Entre mayor es $\pi(x)$ mayor tranquilidad para asignar la clase 1 al elemento $x$ en el espacio se entrada.
2. Si $\pi(x)$ es menor que 0.5, la clase que debe asigna es 0. Entre mayor es $\pi(x)$ mayor tranquilidad para asignar la clase 0 al elemento $x$ en el espacio se entrada.
3. Si $\pi(x)=0.5$, no se puede asignar una clase. Para valores muy cercanos a 0.5, no se debe asignar una clase directamente. Si fuera necesario tomar una decisión, lo mejor es seleccionar la clase de forma aleatoria. Como regla de combate, si $0.48 \le \pi(x)\le 0.52$, seleccionar aleatoriamente.

## <span style="color:blue">Importar los módulos requeridos</span>

In [185]:
#importing alll the necessary packages to Logistic Regression 
# !pip install --upgrade jax jaxlib 
from __future__ import print_function
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
from jax.scipy.special import expit
key = random.PRNGKey(0)
# Current convention is to import original numpy as "onp"
import numpy as onp
import itertools
#import random
#import jaxhm
from sklearn import metrics #for checking the model accuracy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
import seaborn as sb

## <span style="color:blue">Función de predicción</span>

In [196]:
def sigmoid(x):
    return 0.5*(np.tanh(x/2)+1)

# outputs probability of a label being true
def predict(W,b,inputs):
    return sigmoid(np.dot(inputs,W)+b)

## <span style="color:blue">Función de perdida. Entropía cruzada</span>

In [197]:
# training loss: -log likelihood of trainig examples
def loss(W,b,x,y):
    preds = predict(W,b,x)
    label_probs = preds*y + (1-preds)*(1-y)
    return -np.sum(np.log(label_probs))

# initialize coefficients
key, W_key, b_key = random.split(key,3)
W = random.normal(key, (4,))
b = random.normal(key,())

## <span style="color:blue">Carga del conjunto de datos Iris</span>

In [198]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
Y = iris.target

In [199]:
# Definición de nombres de columnas y dimensiones objetivo
col_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
target_dimensions = ['0', '1', '2']

# Crear un DataFrame de Pandas con los datos
iris_df = pd.DataFrame(data=X, columns=col_names[:-1])  # Excluimos 'Species'
iris_df['Species'] = Y  # Agregamos la columna 'Species'

# Mapear los valores de 'Species' a las dimensiones objetivo
iris_df['Species'] = iris_df['Species'].map({0: target_dimensions[0], 1: target_dimensions[1], 2: target_dimensions[2]})

In [200]:
species_mapping = {"0": 0, "1": 1, "2": 2}
iris_df['Species'] = iris_df['Species'].map(species_mapping)

In [201]:
# Dividir el DataFrame en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(iris_df, test_size=30, train_size=120, stratify=iris_df['Species'], random_state=42)

In [202]:
# esta sección es para omitir la clase 0: "Setosa" y recodificar loa datos  de entrenamiento
train_df = train_df[train_df['Species'] >= 1]
#training['Species'] = training['Species'].replace([1,2], [0,1])
train_df['Species'].replace(to_replace=[1,2], value=[0,1], inplace=True)

# esta sección es para omitir la clase 0: "Setosa" y recodificar los datos  de validación
test_df = test_df[test_df['Species'] >= 1]
#test['Species'] = test['Species'].replace([1,2], [0,1])
test_df['Species'].replace(to_replace=[1,2], value=[0,1], inplace=True)

In [203]:
# omite los índices de los dos dataframes para poderlos concadenar
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# concadena los dataframes
iris_dataframe = pd.concat([train_df, test_df], axis=0)

In [204]:
iris_dataframe

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,4.9,2.5,4.5,1.7,1
1,6.8,2.8,4.8,1.4,0
2,5.5,2.5,4.0,1.3,0
3,6.3,2.5,5.0,1.9,1
4,5.6,2.7,4.2,1.3,0
...,...,...,...,...,...
15,6.1,2.6,5.6,1.4,1
16,6.4,2.8,5.6,2.2,1
17,6.7,3.0,5.0,1.7,0
18,6.6,3.0,4.4,1.4,0


In [205]:
targets = iris_dataframe.iloc[:, -1].astype(bool)
targets = np.array(targets)
# Convertir el DataFrame a un Array de JAX
inputs = np.array(iris_dataframe.iloc[:, :-1])  # Excluye la última columna (labels)

In [206]:
targets

Array([ True, False, False,  True, False,  True,  True,  True,  True,
       False, False, False, False,  True,  True, False,  True, False,
        True,  True,  True, False, False,  True,  True, False, False,
        True, False, False,  True,  True, False,  True, False,  True,
       False,  True,  True, False,  True, False, False,  True, False,
       False,  True,  True, False,  True, False,  True,  True, False,
        True, False,  True,  True, False, False, False,  True,  True,
        True,  True, False,  True,  True, False,  True,  True, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False,  True, False,  True,  True,  True, False,
       False, False,  True,  True, False,  True,  True, False, False,
        True], dtype=bool)

In [207]:
inputs

Array([[4.9, 2.5, 4.5, 1.7],
       [6.8, 2.8, 4.8, 1.4],
       [5.5, 2.5, 4. , 1.3],
       [6.3, 2.5, 5. , 1.9],
       [5.6, 2.7, 4.2, 1.3],
       [6.3, 2.8, 5.1, 1.5],
       [7.7, 3. , 6.1, 2.3],
       [7.7, 3.8, 6.7, 2.2],
       [7.6, 3. , 6.6, 2.1],
       [6. , 2.9, 4.5, 1.5],
       [5. , 2. , 3.5, 1. ],
       [5.8, 2.7, 4.1, 1. ],
       [5.8, 2.6, 4. , 1.2],
       [6.7, 2.5, 5.8, 1.8],
       [6.4, 3.1, 5.5, 1.8],
       [6.3, 2.5, 4.9, 1.5],
       [7.9, 3.8, 6.4, 2. ],
       [5.5, 2.4, 3.7, 1. ],
       [6.3, 2.9, 5.6, 1.8],
       [6.4, 2.7, 5.3, 1.9],
       [5.6, 2.8, 4.9, 2. ],
       [6.1, 2.8, 4. , 1.3],
       [6.1, 3. , 4.6, 1.4],
       [6.9, 3.1, 5.4, 2.1],
       [6.2, 3.4, 5.4, 2.3],
       [5.7, 2.6, 3.5, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [6.7, 3.3, 5.7, 2.5],
       [5.9, 3.2, 4.8, 1.8],
       [7. , 3.2, 4.7, 1.4],
       [7.4, 2.8, 6.1, 1.9],
       [5.8, 2.7, 5.1, 1.9],
       [5.6, 2.9, 3.6, 1.3],
       [6.3, 3.3, 6. , 2.5],
       [5.7, 2

In [208]:
def minmax_scale(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    scaled_arr = (arr - min_val) / (max_val - min_val)
    return scaled_arr

# Reescalar los datos en el rango de 0 a 1
scaled_inputs = minmax_scale(inputs)

In [209]:
scaled_inputs

Array([[0.5652174 , 0.2173913 , 0.5072464 , 0.10144928],
       [0.84057975, 0.26086956, 0.5507247 , 0.05797101],
       [0.6521739 , 0.2173913 , 0.4347826 , 0.04347825],
       [0.76811594, 0.2173913 , 0.5797101 , 0.13043478],
       [0.6666666 , 0.24637681, 0.4637681 , 0.04347825],
       [0.76811594, 0.26086956, 0.5942029 , 0.07246377],
       [0.97101444, 0.28985506, 0.73913044, 0.18840578],
       [0.97101444, 0.4057971 , 0.82608694, 0.17391305],
       [0.9565217 , 0.28985506, 0.8115942 , 0.15942027],
       [0.7246377 , 0.27536234, 0.5072464 , 0.07246377],
       [0.5797101 , 0.14492753, 0.36231884, 0.        ],
       [0.6956522 , 0.24637681, 0.44927534, 0.        ],
       [0.6956522 , 0.23188405, 0.4347826 , 0.02898551],
       [0.82608694, 0.2173913 , 0.6956522 , 0.11594202],
       [0.7826087 , 0.3043478 , 0.6521739 , 0.11594202],
       [0.76811594, 0.2173913 , 0.5652174 , 0.07246377],
       [1.        , 0.4057971 , 0.7826087 , 0.14492753],
       [0.6521739 , 0.20289856,

## <span style="color:blue">Gradiente</span>

In [210]:
# compile with jit
# argsnums define positional params to derive with respect to
grad_loss = jit(grad(loss,argnums=(0,1)))

In [211]:
W_grad, b_grad = grad_loss(W,b,scaled_inputs, targets)
print("W_grad = ", W_grad)
print("b_grad = ", b_grad)

W_grad =  [25.351105   9.120643  15.936953   1.0731666]
b_grad =  36.18766


## <span style="color:blue">Entrenamiento del modelo</span>

In [212]:
# train function
def train(W,b,x,y, lr= 0.12):
    gradient = grad_loss(W,b,scaled_inputs,targets) 
    W_grad, b_grad = grad_loss(W,b,scaled_inputs,targets)
    W -= W_grad*lr
    b -= b_grad*lr
    return(W,b)

In [213]:
#    
weights, biases = [], []
train_loss= []
epochs = 20

train_loss.append(loss(W,b,scaled_inputs,targets))

for epoch in range(epochs):
    W,b = train(W,b,scaled_inputs, targets)
    weights.append(W)
    biases.append(b)
    losss = loss(W,b,scaled_inputs,targets)
    train_loss.append(losss)
    print(f"Epoch {epoch}: train loss {losss}")

Epoch 0: train loss 325.0141296386719
Epoch 1: train loss 292.13348388671875
Epoch 2: train loss 258.27178955078125
Epoch 3: train loss 337.6695556640625
Epoch 4: train loss 199.40322875976562
Epoch 5: train loss 368.65875244140625
Epoch 6: train loss 156.45321655273438
Epoch 7: train loss 375.7554016113281
Epoch 8: train loss 138.81214904785156
Epoch 9: train loss 367.92999267578125
Epoch 10: train loss 136.29612731933594
Epoch 11: train loss 360.12176513671875
Epoch 12: train loss 133.8798828125
Epoch 13: train loss 351.67352294921875
Epoch 14: train loss 132.2113037109375
Epoch 15: train loss 342.7821350097656
Epoch 16: train loss 131.06710815429688
Epoch 17: train loss 333.46978759765625
Epoch 18: train loss 130.40155029296875
Epoch 19: train loss 323.79229736328125


In [214]:
print('weights')
for weight in weights:
    print(weight)
print('biases')
for bias in biases:
    print(bias)

weights
[-2.0793319  -0.93917024 -2.1738777   0.770107  ]
[2.7608976  0.77061915 1.7708406  1.660217  ]
[-1.5116806  -0.76155317 -1.0500882   1.3786883 ]
[3.288805  0.9340483 2.865218  2.2637231]
[-0.9959121  -0.60244346  0.03559518  1.9809015 ]
[3.6984477 1.0553026 3.8707294 2.851115 ]
[-0.58982134 -0.48245192  1.0385962   2.567941  ]
[3.9127417 1.1070329 4.725971  3.4092064]
[-0.37598038 -0.4308877   1.8934948   3.1259727 ]
[4.0030913 1.1148568 5.482497  3.9462106]
[-0.2850442  -0.42285812  2.6503806   3.6629958 ]
[4.0913477 1.1222001 6.233519  4.480039 ]
[-0.19597435 -0.41523385  3.4019005   4.196852  ]
[4.1721125 1.1272035 6.9750404 5.010052 ]
[-0.11415768 -0.40987027  4.144059    4.7268963 ]
[4.247022  1.1304967 7.708536  5.536704 ]
[-0.03782129 -0.40608764  4.8784223   5.2535934 ]
[4.316253  1.1321818 8.434326  6.0602074]
[ 0.03334808 -0.40373707  5.6053925   5.7771587 ]
[4.380256  1.1324477 9.152916  6.5807924]
biases
-3.223681
2.750286
-3.2223687
2.6986742
-3.2907038
2.4923015


In [215]:
print(grad(loss)(W,b,scaled_inputs,targets))

[35.66957   12.7920265 23.561605   2.3580925]


## <span style="color:blue">Calculando el valor de la función y el gradiente con value_and_grad</span>

In [216]:
from jax import value_and_grad
loss_val, Wb_grad = value_and_grad(loss,(0,1))(W,b,scaled_inputs, targets)
print('loss value: ', loss_val)
print('gradient value: ', Wb_grad)

loss value:  323.7923
gradient value:  (Array([35.66957  , 12.7920265, 23.561605 ,  2.3580925], dtype=float32), Array(49.84816, dtype=float32))


-[Regresar al inicio](#Contenido)